Things to add:
- verify the correctness of the minimax algorithm (sacrificed the queen), setup a smaller situation (maybe not in chess, a much simpler situation)
    - the minimax algorithm is working based on our tests, so the strange behaviour is from somewhere else probably
- more in-depth board scoring function
    - made up of several smaller scoring functions
    - randomly chooses between moves with a similar score
- with the more in-depth scoring function, add explainability to each board instance (give commentary on each move that was made)
    - adding a printout after the board is printed
    - similar to that one paper, RGA+

In [2]:
import chess
import random
from math import inf
from IPython.display import display, clear_output

MAX_DEPTH = 4
rand = random.Random(0)

In [3]:
SCORES = {
    chess.KING : 200,
    chess.QUEEN : 250,
    chess.ROOK : 5,
    chess.BISHOP : 3,
    chess.KNIGHT : 3,
    chess.PAWN : 1,
}
# Scoring function based on the combined weighted score of multiple smaller scoring functions
def evaluation(b: chess.Board):
    material_score = eval_material(b)

    return material_score

# Very basic eval function based on counting pieces on the board
def eval_material(b:chess.Board):
    if b.is_checkmate():
        return -inf if b.turn == chess.WHITE else inf
    
    # The list needs to be size 7 because PIECE_TYPES goes from 1 to 6
    piece_count = { c : [0] * 7 for c in chess.COLORS }
    for piece in b.piece_map().values():
        piece_count[piece.color][piece.piece_type] += 1

    total = 0
    for piece in chess.PIECE_TYPES:
        diff = piece_count[chess.WHITE][piece] - piece_count[chess.BLACK][piece]
        total += SCORES[piece] * diff
    
    return total


In [4]:
def get_alpha_beta_move(board:chess.Board, player):
    opponent = not player

    def max_value(b:chess.Board, alpha, beta, depth):
        e = evaluation(b)
        if depth == MAX_DEPTH or abs(e) == inf:
            return e, None
        
        same_score_moves = []
        v = -inf
       
        for m in b.legal_moves:
            temp = b.copy()
            temp.push(m)
            s_val = min_value(temp, alpha, beta, depth + 1)
            # printout_str = (f"move = {m}, s_val = {s_val}")
            if s_val > v:
                v = s_val
                same_score_moves = [m]
            elif s_val == v:
                same_score_moves.append(m)
            if v >= beta:
                return v, move
            alpha = max(alpha, v)
        move = random.choice(same_score_moves)
        return v, move

    def min_value(b:chess.Board, alpha, beta, depth):
        e = evaluation(b)
        if depth == MAX_DEPTH or abs(e) == inf:
            return e
        
        same_score_moves = []
        v = inf
        for m in b.legal_moves:
            temp = b.copy()
            temp.push(m)
            v = min(v, max_value(temp, alpha, beta, depth + 1)[0])
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    return max_value(board, -inf, inf, 0)[1]

In [5]:
def get_expectimax_move(board:chess.Board, player):
    opponent = not player

    def max_value(b:chess.Board, depth:int):
        e = evaluation(b)
        if depth == MAX_DEPTH or abs(e) == inf:
            return e, None
        
        v = -inf
        move = None
        for m in b.legal_moves:
            temp = b.copy()
            temp.push(m)
            s_val = chance_value(temp, depth + 1)
            if s_val > v:
                v, move = s_val, m
        return v, move

    def chance_value(b:chess.Board, depth:int):
        e = evaluation(b)
        if depth == MAX_DEPTH or abs(e) == inf:
            return e, None
        
        successors = []
        for m in b.legal_moves:
            temp = b.copy()
            temp.push(m)
            successors.append(max_value(temp, depth + 1)[0])
        return sum(successors) / len(successors)
    
    return max_value(board, 0)[1]

def get_random_move(b:chess.Board, player):
    return rand.choice(list(b.legal_moves))

In [6]:
def play_game(p1, p2):
    """
    Play a game of chess

    Parameters p1 and p2 represent the white and black players respectively

    They should be one of:

    get_alpha_beta_move,
    get_expectimax_move,
    get_random_move

    Turns must be progressed manually by pressing Enter when prompted
    
    Enter "q" to stop the game
    """
    board = chess.Board()
    display(board)
    current_player = p1

    while not board.is_checkmate():
        move = current_player(board, board.turn)
        board.push(move)
        clear_output(True)
        display(board)

        print("=== Commentary ===")
        print("dummy text")
        
        current_player = p1 if board.turn == chess.WHITE else p2
        if input() == "q":
            break

In [ ]:
play_game(get_alpha_beta_move, get_random_move)

In [ ]:
play_game(get_expectimax_move, get_random_move)

In [4]:
def get_alpha_beta_move_tree(root:Node, player:bool):
    opponent = not player

    def max_value(node:Node, depth):
        if depth == MAX_DEPTH:
            return node.cost, None
        
        v = -inf
        move = None
        for m in node.connected_nodes:
            s_val = min_value(m, depth + 1)
            if s_val > v:
                v, move = s_val, m
        return v, move

    def min_value(node:Node, depth):
        if depth == MAX_DEPTH:
            return node.cost
        
        v = inf
        for m in node.connected_nodes:
            s_val = max_value(m, depth + 1)[0]
            if s_val < v:
                v = s_val
        return v

    return max_value(root, 0)[1]

In [ ]:
a = Node('a', 0)
b = Node('b', 0)
c = Node('c', 0)
d = Node('d', 0)
e = Node('e', 0)
f = Node('f', 0)
g = Node('g', 0)
h = Node('h', 2)
i = Node('i', 3)
j = Node('j', 6)
k = Node('k', 1)
l = Node('l', 9)
m = Node('m', 2)
n = Node('n', 8)
o = Node('o', 5)

a.connected_nodes = [b, c]
b.connected_nodes = [d, e]
c.connected_nodes = [f, g]
d.connected_nodes = [h, i]
e.connected_nodes = [j, k]
f.connected_nodes = [l, m]
g.connected_nodes = [n, o]

MAX_DEPTH = 1
inf = 10000000000000000

print(get_alpha_beta_move_tree(g, True).id)

In [7]:
class Node:
    """
    This class describes a single node contained within a graph.
    It has the following instannce level attributes:

    ID: An integer id for the node i.e. 1
    """
    def __init__(self, id, cost):
        self.id = id
        self.cost = cost
        self.connected_nodes = []